In [4]:
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
from PIL import Image
import os

In [5]:
file = "test_data/Shannon_20_Fullres"
hdr_file = f"{file}.hdr"
# Check if files exist
if os.path.exists(file):
    print(f"File exists: {file}")
else:
    print(f"File not found: {file}")

if os.path.exists(hdr_file):
    print(f"HDR File exists: {hdr_file}")
else:
    print(f"HDR File not found: {hdr_file}")

File exists: test_data/Shannon_20_Fullres
HDR File exists: test_data/Shannon_20_Fullres.hdr


In [ ]:
# Open the ENVI file
img_open = gdal.Open(file)

# Read as NumPy array
shannon_diversity_map = img_open.ReadAsArray()

# Check raster size
print(f"Original raster size: {shannon_diversity_map.shape}")

# Handle NoData values (replace -9999 with NaN)
shannon_diversity_map = np.where(shannon_diversity_map == -9999, np.nan, shannon_diversity_map)
#shannon_diversity_map = np.where(shannon_diversity_map == 0, np.nan, shannon_diversity_map)

# Debug: Check min/max values
print(f"Min value: {np.nanmin(shannon_diversity_map)}, Max value: {np.nanmax(shannon_diversity_map)}")

# If all values are NaN, exit
if np.isnan(shannon_diversity_map).all():
    raise ValueError("Error: The entire raster contains NaN values. Check your input file!")

# Normalize to 0-255
min_val, max_val = np.nanmin(shannon_diversity_map), np.nanmax(shannon_diversity_map)
normalized_map = (shannon_diversity_map - min_val) / (max_val - min_val) * 255

# Convert to uint8, setting NaNs to 0 (black)
normalized_map = np.nan_to_num(normalized_map, nan=0).astype(np.uint8)

# Apply colormap (same as before)
colormap = plt.get_cmap("turbo")
colored_image = colormap(normalized_map / 255.0)

# --- Key Change: Convert RGBA to RGB *before* handling NaN values ---
# This is the most important change!
colored_image = (colored_image[:, :, :3] * 255).astype(np.uint8)
#colored_image = (colored_image * 255).astype(np.uint8)  # Keep RGBA

# Handle NaNs *after* converting to RGB
# This ensures that NaNs are handled correctly in the RGB image.
nan_mask = np.isnan(shannon_diversity_map)  # Create a mask of NaN values
colored_image[nan_mask] = [0, 0, 0] # Set NaN pixels to black (or any color you want)

# --- End of key change ---


# Save using PIL (no changes needed here, but good practice to include mode)
Image.fromarray(colored_image, mode="RGB").save(output_png)  # Specify mode="RGB"

print(f"Saved PNG: {output_png}")


Original raster size: (1020, 1020)
Min value: 0.0, Max value: 2.3112242221832275


ValueError: shape mismatch: value array of shape (3,) could not be broadcast to indexing result of shape (630000,4)